In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np 
import geopandas as gpd
import geoplot as gplt
import geoplot.crs as gcrs

## Explore and Clean

In [ ]:
#read in Intake DF CSV
intake_df = pd.read_csv('../data/intake_df.csv')

In [ ]:
# reorganize results of table and place relevant columns
fruit_intake_df = intake_df[['state','sample_size','pct_rec_f_in','med_f_in_freq']]
# remove parenthesis number range in string
fruit_intake_df['pct_rec_f_in'] = intake_df['pct_rec_f_in'].str[0:4]
# convert to float
fruit_intake_df['pct_rec_f_in'] = fruit_intake_df['pct_rec_f_in'].astype(float)
# more floats
fruit_intake_df['med_f_in_freq'] = fruit_intake_df['med_f_in_freq'].astype(float)
# string fix to int
fruit_intake_df['sample_size'] = fruit_intake_df['sample_size'].str.replace(',','')
fruit_intake_df['sample_size'] = fruit_intake_df['sample_size'].astype(int)

In [ ]:
# bring in geodata for maps
state_lines = gpd.read_file('/home/nhac/Documents/NSS/Python/projects/fruit_zones_capstone/data/gz_2010_us_040_00_500k.json')
hardiness_zone = gpd.read_file('/home/nhac/Documents/NSS/Python/projects/fruit_zones_capstone/data/us-hardiness-zones_1291.geojson')

In [ ]:
# add column for geodataframe merge
state_lines['state'] = state_lines['NAME']

In [ ]:
# get lowest percent states
low_pct = fruit_intake_df.sort_values(by='pct_rec_f_in')
# maybe need these 5 for closer comparison later
low_five = low_pct.head()
low_five_geo = low_five.merge(state_lines, on='state', how='inner')

In [ ]:
# create geodataframe from the lowest five states
low_five_geo = gpd.GeoDataFrame(low_five_geo, geometry='geometry')

In [ ]:
low_pct.head()

In [ ]:
# map lowest 5, did not need above geoDF...
ax = low_pct.plot.bar(figsize=(11,6), 
                      y='pct_rec_f_in', 
                      x='state', 
                      color='#cccccc',
                      rot=90)
plt.title('Percentage of Population Meeting Recommended Fruit Intake',
          fontdict={'fontsize':16},
          color='#4C4C4C',
          weight='bold')
plt.xlabel('State', 
           fontdict={'fontsize':14}, 
           color='#2499BE')
plt.ylabel('Percentage', 
           fontdict={'fontsize':14}, 
           color='#2499BE')
plt.xticks(rotation=90, 
           size=11)
ax.legend(['% of Population'])
#ax.tick_params(axis='both', which='major', labelsize=9)
ax.set_facecolor('#2499BE')
lowest = ax.patches     
for i in range(10):
    lowest[i].set_facecolor("#4f4f4f")   # highlight the lowest 10 states

In [ ]:
# create geoDF for fruit intake choropleth
fruit_intake_gdf = pd.merge(fruit_intake_df,state_lines, on='state', how='inner')

In [ ]:
# get rid of hawaii and alaska for contiguous USA only
fruit_intake_gdf = fruit_intake_gdf[~fruit_intake_gdf["state"].isin(["Hawaii", "Alaska"])]
# make it geo
fruit_intake_gdf = gpd.GeoDataFrame(fruit_intake_gdf, geometry='geometry')
# make sure its epsg4326
fruit_intake_gdf = fruit_intake_gdf.to_crs("EPSG:4326")

In [ ]:
#fruit_intake_gdf.to_csv('../data/geo_fruit_intake.csv') 

In [ ]:
# map it using geoplot
import mapclassify   # use this for quantile separation for visual clarity

# set min and max lon and lat for USA map extents (zoom in)
min_lon, max_lon = -125, -65
min_lat, max_lat = 24, 50
# plot map

ax = gplt.choropleth(fruit_intake_gdf,
                   hue="pct_rec_f_in",
                   #scheme=mapclassify.Quantiles(fruit_intake_gdf["pct_rec_f_in"], k=4),
                   cmap="gist_earth_r", #edgecolor="grey",
                   extent=(-172,10, 10,75),
                   projection=gcrs.WebMercator(),
                   #legend_labels=["Least Met Recommended", "Somewhat Met", "Better","Best"],
                   #legend_kwargs={"loc":"best",
                   #               "fontsize": "small",
                   #              "title":"% of Sample Size",
                   #               "title_fontsize":"medium"},
                   figsize=(8,6), legend=True);

gplt.polyplot(state_lines, 
              linewidth=0.8,
              ax=ax
)


ax.set_extent((min_lon, max_lon, min_lat, max_lat))
ax.set_facecolor('#2499BE')
#ax.set_axis_bgcolor("#2499BE")

plt.title("Percent of Population Meeting Recommended Intake", 
          fontdict={"fontsize": 16}, 
          pad=12,
          color='#4c4c4c');

#make this a color gradient

In [ ]:
# set min and max lon and lat for USA map extents (zoom in)
min_lon, max_lon = -125, -65
min_lat, max_lat = 24, 50
# plot map
ax = gplt.choropleth(fruit_intake_gdf,
                   hue="pct_rec_f_in",
                   scheme=mapclassify.Quantiles(fruit_intake_gdf["pct_rec_f_in"], k=4),
                   cmap="Blues", edgecolor="grey",
                   extent=(-172,10, 10,75),
                   projection=gcrs.WebMercator(),
                   #legend_labels=["Least Met Recommended", "Somewhat Met", "Better","Best"],
                   legend_kwargs={"loc":"best",
                                  "fontsize": "small",
                                 "title":"% of Sample Size",
                                  "title_fontsize":"medium"},
                   figsize=(8,6), legend=True);

#gplt.polyplot(state_lines, 
#              linewidth=0.5,
#              ax=ax
#)

ax.set_extent((min_lon, max_lon, min_lat, max_lat))
ax.set_facecolor('lightgrey')
plt.title("Percent of Population Meeting Recommended Intake", fontdict={"fontsize": 14}, pad=5);

## Health data and demographics

In [ ]:
# bring stuff in
age_less = pd.read_csv('../data/age-pct_adults_less_than_1x_day.csv')
edu_less = pd.read_csv('../data/edu-pct_adults_less_than_1x_day.csv')
ob_edu_2011 = pd.read_csv('../data/edu_obesity_2011.csv')
ob_edu_2022 = pd.read_csv('../data/edu_obesity_2022_.csv')
ob_total_2011 = pd.read_csv('../data/obesity_2011_total.csv')
ob_total_2020 = pd.read_csv('../data/obesity_2020_total.csv')

In [ ]:
# function to view column names in a dataframe
def getColumns(df):
    for col in df.columns:
        print(col)

In [ ]:
# trim columns for easier manipulation (fruit intake less than 1x per day)
trim_age_less = age_less[['Stratification1','Data_Value']]
trim_edu_less = edu_less[['Stratification1','Data_Value']]

In [ ]:
# national adult obesity

In [ ]:
ob_tot_2011_drop = ob_total_2011[ob_total_2011['LocationDesc'].notna()]
ob_tot_2011_drop = ob_tot_2011_drop.loc[ob_tot_2011_drop['LocationDesc'] != 'National']

In [ ]:
ob_tot_2020_drop = ob_total_2020[ob_total_2020['LocationDesc'].notna()]
ob_tot_2020_drop = ob_tot_2020_drop.loc[ob_tot_2020_drop['LocationDesc'] != 'National']

In [ ]:
ob_tot_2011_drop=ob_tot_2011_drop.sort_values(by='Data_Value', ascending=False)

### Graph

In [ ]:
ax = ob_tot_2011_drop.plot.bar(x='LocationDesc', y='Data_Value', label='percent', figsize=(8,6))
#ob_tot_2020_drop.plot(ax = ax, x='LocationAbbr', y='Data_Value', label='2020')
plt.title('National Percentage of Adult Obesity (2011)')
plt.legend(title='Percentage of State Population')
highest = ax.patches     
for i in range(10):
    highest[i].set_facecolor("#A2052B")   # highlight the lowest 10 states
plt.show()

In [ ]:
ob_tot_2020_drop=ob_tot_2020_drop.sort_values(by='Data_Value', ascending=False)

### Graph

In [ ]:
ax = ob_tot_2020_drop.plot.bar(x='LocationDesc', 
                               y='Data_Value', 
                               label='State Percentage', 
                               figsize=(11,6),
                               color='#cccccc')

#ob_tot_2020_drop.plot(ax = ax, x='LocationAbbr', y='Data_Value', label='2020')
plt.title('National Percentage of Adult Obesity by State (2020)',
          fontdict={'fontsize':16},
          color='#4C4C4C',
          weight='bold')
plt.xlabel("States", 
           fontdict={'fontsize':14}, 
           color='#2499BE')
plt.xticks(rotation=90, 
           size=11)
plt.ylabel("National Percentage", fontdict={'fontsize':14}, color='#2499BE')
plt.legend(title='Percentage of State Population')
highest = ax.patches     
for i in range(10):
    highest[i].set_facecolor("#4f4f4f") # highlight the lowest 10 states
ax.set_facecolor('#2499BE')
    
plt.show()

In [ ]:
ob_tot_2011_drop = ob_tot_2011_drop.filter(['LocationDesc','LocationAbbr','Data_Value','GeoLocation_Lat','GeoLocation_Long'], axis=1)
ob_tot_2020_drop = ob_tot_2020_drop.filter(['LocationDesc','LocationAbbr','Data_Value','GeoLocation_Lat','GeoLocation_Long'], axis=1)

In [ ]:
#obesity education level

In [ ]:
ob_edu_trim22 = ob_edu_2022[['LocationDesc','Stratification1','Data_Value']]
ob_edu_national22 = ob_edu_trim22.loc[ob_edu_trim22['LocationDesc'] == 'National']

In [ ]:
ob_edu_trim11 = ob_edu_2011[['LocationDesc','Stratification1','Data_Value']]
ob_edu_national11 = ob_edu_trim11.loc[ob_edu_trim11['LocationDesc'] == 'National']

### Graph

In [ ]:
plt.rcParams["figure.figsize"] = [10, 3.50]
plt.rcParams["figure.autolayout"] = True

ax = ob_edu_national11.plot(x='Stratification1', y='Data_Value', label='2011')
ob_edu_national22.plot(ax = ax, x='Stratification1', y='Data_Value', label='2022')
plt.title('National Percentage of Adult Obesity by Education Level (2011 vs 2022)')
plt.show()

In [ ]:
X = ob_edu_national11['Stratification1']
Y11 = ob_edu_national11['Data_Value']
Z22 = ob_edu_national22['Data_Value']
X_axis = np.arange(len(X))

plt.figure(facecolor='#0F9BBC', figsize=(9,5))
ax = plt.axes()
ax.set_facecolor("#0F9BBC")

plt.bar(X_axis - 0.2, Y11, 0.4, label = '2011', color='#ffffee')
plt.bar(X_axis + 0.2, Z22, 0.4, label = '2022', color='darkgrey')
plt.grid(axis='y', linestyle = '--')
plt.xticks(X_axis, X, rotation=27.5, color='white')
plt.yticks(color='white')
plt.xlabel("Education Level", 
           color='#0f0f0f', 
           fontdict={"fontsize": 14}, 
           labelpad=10)
plt.ylabel("National Percentage", 
           color='#0f0f0f', 
           fontdict={"fontsize": 14},
           labelpad=6)
plt.title("Percentage of Obesity in Adults by Level of Education", 
          color='white', 
          weight='bold', 
          fontdict={"fontsize": 16},
          pad=10)
plt.legend(title='Year')
plt.show()
#ax.bar(ob_edu_national11['Stratification1'], ob_edu_national11['Data_Value'], width=0.2, color='b')
#ax.bar(ob_edu_national22['Stratification1'], ob_edu_national22['Data_Value'], width=0.2, color='r')


In [ ]:
import seaborn as sns

In [ ]:
ob_tot_2020_drop = ob_tot_2020_drop[~ob_tot_2020_drop["LocationDesc"].isin(["Guam", "Puerto Rico"])]